# Imports

In [313]:
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import requests
import time
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from fake_useragent import UserAgent
import time

# Log into Consumer Reports

In [353]:
DRIVER_PATH = '/Users/blantj/documents/project/car_sales/chromedriver'
options = Options()
options.headless = True
url = 'https://secure.consumerreports.org/ec/login'
options.add_argument("--window-size=1920,1200")
driver = webdriver.Chrome(options=options, executable_path=DRIVER_PATH)
driver.get(url)
login = driver.find_element_by_id("username")
login.send_keys('iblant@yahoo.com')
password = driver.find_element_by_id("password")
password.send_keys('Tigers24')
submit = driver.find_element_by_tag_name("button").click()


# Scrape Consumer Reports Review URLs

In [315]:
ranking_urls = ['https://www.consumerreports.org/cars/types/new/hybrids-evs/ratings',
              'https://www.consumerreports.org/cars/types/new/luxury-cars/ratings',
             'https://www.consumerreports.org/cars/types/new/minivans/ratings',
             'https://www.consumerreports.org/cars/types/new/pickup-trucks/ratings',
             'https://www.consumerreports.org/cars/types/new/sedans/ratings',
             'https://www.consumerreports.org/cars/types/new/small-cars/ratings',
             'https://www.consumerreports.org/cars/types/new/sports-cars/ratings',
             'https://www.consumerreports.org/cars/types/new/suvs/ratings',
             'https://www.consumerreports.org/cars/types/new/wagons/ratings']
review_urls = []
for url in ranking_urls:
    driver.get(url)
    time.sleep(.2)
    review_urls_parent_1 = driver.find_element_by_xpath('/html/body/div[5]/div/div[2]/div/div[4]/table/tbody')
    review_urls_parent_2 = review_urls_parent_1.find_elements_by_tag_name('tr')
    review_urls_parent_3 = []
    for parent_element in review_urls_parent_2:
        review_urls_parent_3.extend(parent_element.find_elements_by_xpath('td[2]/div/div[1]/div[1]/a'))
    for parent_element in review_urls_parent_3:
        review_urls.append(parent_element.get_attribute('href'))
    print(url)

https://www.consumerreports.org/cars/types/new/hybrids-evs/ratings
https://www.consumerreports.org/cars/types/new/luxury-cars/ratings
https://www.consumerreports.org/cars/types/new/minivans/ratings
https://www.consumerreports.org/cars/types/new/pickup-trucks/ratings
https://www.consumerreports.org/cars/types/new/sedans/ratings
https://www.consumerreports.org/cars/types/new/small-cars/ratings
https://www.consumerreports.org/cars/types/new/sports-cars/ratings
https://www.consumerreports.org/cars/types/new/suvs/ratings
https://www.consumerreports.org/cars/types/new/wagons/ratings


In [316]:
used_ranking_urls = []
for url in ranking_urls:
    url = url.replace('ratings','')
    used_ranking_urls.append(url.replace('new','used'))
used_review_urls = []
for url in used_ranking_urls:
    driver.get(url)
    time.sleep(.2)
    used_review_urls_parent_1 = driver.find_elements_by_partial_link_text('201')
    for parent_element in used_review_urls_parent_1:
        used_review_urls.append(parent_element.get_attribute('href'))
    print(url)

https://www.consumerreports.org/cars/types/used/hybrids-evs/
https://www.consumerreports.org/cars/types/used/luxury-cars/
https://www.consumerreports.org/cars/types/used/minivans/
https://www.consumerreports.org/cars/types/used/pickup-trucks/
https://www.consumerreports.org/cars/types/used/sedans/
https://www.consumerreports.org/cars/types/used/small-cars/
https://www.consumerreports.org/cars/types/used/sports-cars/
https://www.consumerreports.org/cars/types/used/suvs/
https://www.consumerreports.org/cars/types/used/wagons/


In [317]:
all_review_urls = list(set(review_urls)) + list(set(used_review_urls))
final_review_urls = []
for url in all_review_urls:
    if int(url[-14:-10]) == 2016:
        final_review_urls.append(url.replace('2016','2015'))
        final_review_urls.append(url.replace('2016','2014'))
        final_review_urls.append(url.replace('2016','2013'))
        final_review_urls.append(url.replace('2016','2012'))
        final_review_urls.append(url.replace('2016','2011'))
        final_review_urls.append(url.replace('2016','2010'))
    elif int(url[-14:-10]) == 2015:
        final_review_urls.append(url.replace('2015','2014'))
        final_review_urls.append(url.replace('2015','2013'))
        final_review_urls.append(url.replace('2015','2012'))
        final_review_urls.append(url.replace('2015','2011'))
        final_review_urls.append(url.replace('2015','2010'))
    elif 2021 > int(url[-14:-10]) > 2009:
        final_review_urls.append(url)
final_review_urls = list(set(final_review_urls))
    

# Scrape Consumer Reports Reviews

In [355]:
columns = ('Brand','Model','Year','Overall Score', 'Road Test Score', 'Reliability Score', 'Owner Satisfact Score', 'Warranty',
           'Drivetrain Warranty', 'Corrosion Waranty', 'Roadside Aid', 'Free Maintenance', 'Acceleration Score',
           'Zero To Sixty', 'Transmission Score', 'Fuel Economy Score', 'Overall Fuel Economy', 'City Fuel Economy',
           'Highway Fuel Economy', 'Annual Fuel Cost', 'Braking Score', 'Emergency Handling Score', 'Ride Score',
           'Noise Score', 'Front Seat Score', 'Rear Seat Score', 'Interior Finish Score', 'Trunk Score', 'Seats',
           'Zero To Thirty', 'Fortyfive To Sixty', 'Quarter Mile', 'Quarter Speed', 'Routine Handling Score',
           'Avoidance Max Speed', 'Wet Braking', 'Dry Braking', 'Headlights', 'Driving Position',
           'Front Shoulder Room', 'Front Leg Room', 'Front Head Room', 'Rear Shoulder Room', 'Rear Leg Room',
           'Rear Head Room', 'Third Row Score', 'Third Shoulder Room', 'Third Leg Room', 'Third Head Room',
           'Front Access', 'Rear Access', 'Third Access', 'Usability', 'Climate System', 'Fuel Capacity', 'Fuel Type',
           'Driving Satisfaction', 'Comfort Satisfaction', 'Value Satisfaction', 'Styling Satisfaction')
df = pd.DataFrame(columns=columns)
df.to_csv("Consumer_Reports.csv")

In [366]:
count=0
for review_url in final_review_urls[0:100]:
    driver.get(review_url)
    time.sleep(.2)
    if (len(driver.find_elements_by_xpath('/html/body/div[3]/div/header/div[4]/div/ul/li[1]/a')) > 0) and (len(driver.find_elements_by_class_name('bar-ratings-chart-member__range')) > 0):
        if driver.find_element_by_xpath('/html/body/div[8]/div[1]/div[1]/div[2]/div[1]/div[2]/div/div[3]/div[2]').text!='NA':
            if int(review_url[-14:-10]) == 2020:
                overall_score = driver.find_element_by_class_name('ratings-section-wrapper__header').text[-2:]
                road_test_score = driver.find_element_by_class_name('js-bar-chart-mount').text[0:2]
                reliability_score = driver.find_elements_by_class_name('bar-ratings-chart-member__range')[1].text[0]
                owner_satisfact_score = driver.find_elements_by_class_name('bar-ratings-chart-member__range')[2].text[0]
                warranty = driver.find_element_by_xpath('/html/body/div[11]/div/div/div[3]/div/div[2]/ul/li[1]').text[7:]
                drivetrain_warranty = driver.find_element_by_xpath('/html/body/div[11]/div/div/div[3]/div/div[2]/ul/li[2]').text[12:]
                corrosion_waranty = driver.find_element_by_xpath('/html/body/div[11]/div/div/div[3]/div/div[2]/ul/li[3]').text[11:]
                roadside_aid = driver.find_element_by_xpath('/html/body/div[11]/div/div/div[3]/div/div[2]/ul/li[4]').text[14:]
                free_maintenance = driver.find_element_by_xpath('/html/body/div[11]/div/div/div[3]/div/div[2]/ul/li[5]').text[13:]            
            else:
                overall_score = None
                road_test_score = None
                reliability_score = driver.find_elements_by_class_name('bar-ratings-chart-member__range')[0].text[0]
                owner_satisfact_score = driver.find_elements_by_class_name('bar-ratings-chart-member__range')[1].text[0]
                warranty = None
                drivetrain_warranty = None
                corrosion_waranty = None
                roadside_aid = None
                free_maintenance = None 
            acceleration_score = driver.find_elements_by_class_name('rating-wrapper__data')[0].text[0]
            zero_to_sixty = driver.find_elements_by_class_name('rating-wrapper__data')[1].text[:-4]
            transmission_score = driver.find_elements_by_class_name('display-ratings')[1].text
            fuel_economy_score = driver.find_elements_by_class_name('display-ratings')[2].text
            overal_fuel_economy = driver.find_elements_by_class_name('rating-wrapper__data')[4].text[0:2]
            city_fuel_economy = driver.find_elements_by_class_name('rating-wrapper__data')[5].text[0:2]
            highway_fuel_economy = driver.find_elements_by_class_name('rating-wrapper__data')[6].text[0:2]
            annual_fuel_cost = driver.find_elements_by_class_name('rating-wrapper__data')[7].text[1:]
            braking_score = driver.find_elements_by_class_name('display-ratings')[3].text
            emergency_handling_score = driver.find_elements_by_class_name('display-ratings')[4].text
            ride_score = driver.find_elements_by_class_name('display-ratings')[5].text
            noise_score = driver.find_elements_by_class_name('display-ratings')[6].text
            front_seat_score = driver.find_elements_by_class_name('display-ratings')[7].text
            rear_seat_score = driver.find_element_by_xpath('/html/body/div[8]/div[1]/div[1]/div[2]/div[2]/div[2]/div/div[8]/div[2]/div/div/div/div/div[2]').text[0:1]
            interior_finish_score = driver.find_element_by_xpath('/html/body/div[8]/div[1]/div[1]/div[2]/div[2]/div[2]/div/div[10]/div[2]/div/div/div/div/div[2]').text[0:1]
            trunk_score = driver.find_element_by_xpath('/html/body/div[8]/div[1]/div[1]/div[2]/div[2]/div[2]/div/div[12]/div[2]/div/div/div/div/div[2]').text[0:1]
            seats = driver.find_element_by_xpath('/html/body/div[11]/div/div/div[1]/div/div[2]/div/span').text
            driver.get(review_url.replace('overview','ratings'))
            time.sleep(.2)
            zero_to_thirty = driver.find_element_by_css_selector('div.ratings-collapse:nth-child(2) > div:nth-child(2) > div:nth-child(1) > div:nth-child(1) > div:nth-child(3) > div:nth-child(1) > div:nth-child(1) > div:nth-child(2) > div:nth-child(1)').text[:-5]
            fortyfive_to_sixty = driver.find_element_by_css_selector('div.ratings-collapse:nth-child(2) > div:nth-child(2) > div:nth-child(1) > div:nth-child(1) > div:nth-child(3) > div:nth-child(1) > div:nth-child(3) > div:nth-child(2) > div:nth-child(1)').text[:-5]
            quarter_mile = driver.find_element_by_css_selector('div.ratings-sub-row:nth-child(4) > div:nth-child(2) > div:nth-child(1)').text[:-5]
            quarter_speed = driver.find_element_by_css_selector('div.ratings-sub-row:nth-child(5) > div:nth-child(2) > div:nth-child(1)').text[:-4]
            routine_handling_score = driver.find_element_by_css_selector('div.ratings-collapse:nth-child(2) > div:nth-child(2) > div:nth-child(1) > div:nth-child(3) > div:nth-child(2) > div:nth-child(1) > div:nth-child(1) > div:nth-child(2) > span:nth-child(1)').text[:1]
            avoidance_max_speed = driver.find_element_by_css_selector('div.ratings-collapse:nth-child(2) > div:nth-child(2) > div:nth-child(1) > div:nth-child(4) > div:nth-child(3) > div:nth-child(1) > div:nth-child(1) > div:nth-child(2) > div:nth-child(1)').text[:-4]
            wet_braking = data_point = driver.find_element_by_css_selector('div.ratings-collapse:nth-child(2) > div:nth-child(2) > div:nth-child(1) > div:nth-child(5) > div:nth-child(3) > div:nth-child(1) > div:nth-child(1) > div:nth-child(2) > div:nth-child(1)').text[:-4]
            dry_braking = driver.find_element_by_css_selector('div.ratings-collapse:nth-child(2) > div:nth-child(2) > div:nth-child(1) > div:nth-child(5) > div:nth-child(3) > div:nth-child(1) > div:nth-child(2) > div:nth-child(2) > div:nth-child(1)').text[:-4]
            headlights = driver.find_element_by_css_selector('div.ratings-collapse:nth-child(2) > div:nth-child(2) > div:nth-child(1) > div:nth-child(7) > div:nth-child(2) > div:nth-child(1) > div:nth-child(1) > div:nth-child(2) > span:nth-child(1)').text[:-1]
            driver.find_element_by_xpath('/html/body/div[7]/div/div[2]/div[3]/h3/div[2]').click()
            driving_position = driver.find_element_by_xpath('/html/body/div[7]/div/div[2]/div[3]/div/div/div[3]/div[2]/div/div/div[2]/span[1]').text[:-1]
            front_shoulder_room = driver.find_element_by_xpath('/html/body/div[7]/div/div[2]/div[3]/div/div/div[4]/div[3]/div/div[1]/div[2]/div').text[:-4]
            front_leg_room = driver.find_element_by_xpath('/html/body/div[7]/div/div[2]/div[3]/div/div/div[4]/div[3]/div/div[2]/div[2]/div').text[:-4]
            front_head_room = driver.find_element_by_xpath('/html/body/div[7]/div/div[2]/div[3]/div/div/div[4]/div[3]/div/div[3]/div[2]/div').text[:-4]
            rear_shoulder_room = driver.find_element_by_xpath('/html/body/div[7]/div/div[2]/div[3]/div/div/div[5]/div[3]/div/div[1]/div[2]/div').text[:-4]
            rear_leg_room = driver.find_element_by_xpath('/html/body/div[7]/div/div[2]/div[3]/div/div/div[5]/div[3]/div/div[2]/div[2]/div').text[:-4]
            rear_head_room = driver.find_element_by_xpath('/html/body/div[7]/div/div[2]/div[3]/div/div/div[5]/div[3]/div/div[3]/div[2]/div').text[:-4]
            third_row_score = driver.find_element_by_xpath('/html/body/div[7]/div/div[2]/div[3]/div/div/div[6]/div[2]/div/div/div[2]').text[0:1]
            third_shoulder_room = driver.find_element_by_xpath('/html/body/div[7]/div/div[2]/div[3]/div/div/div[6]/div[3]/div/div[1]/div[2]/div').text[0:-4]
            third_leg_room = driver.find_element_by_xpath('/html/body/div[7]/div/div[2]/div[3]/div/div/div[6]/div[3]/div/div[2]/div[2]/div').text[0:-4]
            third_head_room = driver.find_element_by_xpath('/html/body/div[7]/div/div[2]/div[3]/div/div/div[6]/div[3]/div/div[3]/div[2]/div').text[0:-4]
            front_access = driver.find_element_by_xpath('/html/body/div[7]/div/div[2]/div[3]/div/div/div[7]/div[2]/div/div/div[2]/span[1]').text[0:1]
            rear_access = driver.find_element_by_xpath('/html/body/div[7]/div/div[2]/div[3]/div/div/div[8]/div[2]/div/div/div[2]').text[0:1]
            third_access = driver.find_element_by_xpath('/html/body/div[7]/div/div[2]/div[3]/div/div/div[9]/div[2]/div/div/div[2]').text[0:1]
            usability = driver.find_element_by_xpath('/html/body/div[7]/div/div[2]/div[3]/div/div/div[10]/div[2]/div/div/div[2]/span[1]').text[0:1]
            climate_system = driver.find_element_by_xpath('/html/body/div[7]/div/div[2]/div[3]/div/div/div[13]/div[2]/div/div/div[2]/span[1]').text[0:1]
            driver.find_element_by_xpath('/html/body/div[7]/div/div[2]/div[4]/h3/div[2]').click()
            fuel_capacity = driver.find_element_by_xpath('/html/body/div[7]/div/div[2]/div[4]/div/div/div[6]/div[2]/div').text[0:-5]
            fuel_type = driver.find_element_by_xpath('/html/body/div[7]/div/div[2]/div[4]/div/div/div[7]/div[2]/div').text
            driver.get(review_url.replace('overview','owner-satisfaction'))
            time.sleep(.2)
            driving_satisfaction = driver.find_element_by_xpath('/html/body/div[7]/section/article[1]/div/div/div[2]/div/div/div/div[2]').text[0:2]
            comfort_satisfaction = driver.find_element_by_xpath('/html/body/div[7]/section/article[2]/div/div/div[2]/div/div/div/div[2]').text[0:2]
            value_satisfaction = driver.find_element_by_xpath('/html/body/div[7]/section/article[3]/div/div/div[2]/div/div/div/div[2]').text[0:2]
            styling_satisfaction = driver.find_element_by_xpath('/html/body/div[7]/section/article[4]/div/div/div[2]/div/div/div/div[2]').text[0:2]
            model_year = review_url[-14:-10]
            model_brand = review_url.split('/')[4]
            model_name = review_url.split('/')[5]
            new_df = pd.DataFrame([[model_brand,model_name,model_year,overall_score, road_test_score, reliability_score, owner_satisfact_score, warranty,
                         drivetrain_warranty, corrosion_waranty, roadside_aid, free_maintenance, acceleration_score,
                         zero_to_sixty, transmission_score, fuel_economy_score, overal_fuel_economy, city_fuel_economy,
                         highway_fuel_economy, annual_fuel_cost, braking_score, emergency_handling_score, ride_score,
                         noise_score, front_seat_score, rear_seat_score, interior_finish_score, trunk_score, seats,
                         zero_to_thirty, fortyfive_to_sixty, quarter_mile, quarter_speed, routine_handling_score,
                         avoidance_max_speed, wet_braking, dry_braking, headlights, driving_position,
                         front_shoulder_room, front_leg_room, front_head_room, rear_shoulder_room, rear_leg_room,
                         rear_head_room, third_row_score, third_shoulder_room, third_leg_room, third_head_room,
                         front_access, rear_access, third_access, usability, climate_system, fuel_capacity, fuel_type,
                         driving_satisfaction, comfort_satisfaction, value_satisfaction, styling_satisfaction]], columns=columns)
            old_df = pd.read_csv("Consumer_Reports.csv", index_col = 0)
            df_save = pd.concat([old_df, new_df])
            df_save.to_csv("Consumer_Reports.csv")
            print(count, review_url)
            count += 1
        else:
            print(count, review_url, 'Did not scrape')
            count += 1
    else:
        print(count, review_url, 'Did not scrape')
        count += 1     

0 https://www.consumerreports.org/cars/kia/sportage/2017/overview/
1 https://www.consumerreports.org/cars/chrysler/300/2017/overview/
2 https://www.consumerreports.org/cars/nissan/versa/2020/overview/
3 https://www.consumerreports.org/cars/buick/envision/2014/overview/ Did not scrape
4 https://www.consumerreports.org/cars/volvo/xc90/2012/overview/ Did not scrape
5 https://www.consumerreports.org/cars/ford/f-350/2014/overview/ Did not scrape
6 https://www.consumerreports.org/cars/bmw/x6/2014/overview/ Did not scrape
7 https://www.consumerreports.org/cars/gmc/sierra-1500/2019/overview/
8 https://www.consumerreports.org/cars/bmw/x6/2018/overview/ Did not scrape
9 https://www.consumerreports.org/cars/gmc/sierra-3500hd/2014/overview/ Did not scrape
10 https://www.consumerreports.org/cars/hyundai/genesis/2013/overview/
11 https://www.consumerreports.org/cars/lincoln/corsair/2020/overview/
12 https://www.consumerreports.org/cars/bentley/flying-spur/2010/overview/ Did not scrape
13 https://www

In [362]:
review_url

'https://www.consumerreports.org/cars/land-rover/range-rover/2018/overview/'

In [340]:
driver.get('https://www.consumerreports.org/cars/jaguar/f-type/2020/overview')
time.sleep(.2)
#driver.find_element_by_xpath('/html/body/div[8]/div[1]/div[1]/div[2]/div[1]/div[2]/div/div[3]/div[2]').text
driver.find_elements_by_class_name('bar-ratings-chart-member__range')[1].text[0]




'4'

In [298]:
#Include model and year in scrape

url = 'https://www.consumerreports.org/cars/mazda/cx-9/2020/ratings/'
driver.get(url)
zero_to_thirty = driver.find_element_by_css_selector('div.ratings-collapse:nth-child(2) > div:nth-child(2) > div:nth-child(1) > div:nth-child(1) > div:nth-child(3) > div:nth-child(1) > div:nth-child(1) > div:nth-child(2) > div:nth-child(1)').text[:-5]
fortyfive_to_sixty = driver.find_element_by_css_selector('div.ratings-collapse:nth-child(2) > div:nth-child(2) > div:nth-child(1) > div:nth-child(1) > div:nth-child(3) > div:nth-child(1) > div:nth-child(3) > div:nth-child(2) > div:nth-child(1)').text[:-5]
quarter_mile = driver.find_element_by_css_selector('div.ratings-sub-row:nth-child(4) > div:nth-child(2) > div:nth-child(1)').text[:-5]
quarter_speed = driver.find_element_by_css_selector('div.ratings-sub-row:nth-child(5) > div:nth-child(2) > div:nth-child(1)').text[:-4]
routine_handling_score = driver.find_element_by_css_selector('div.ratings-collapse:nth-child(2) > div:nth-child(2) > div:nth-child(1) > div:nth-child(3) > div:nth-child(2) > div:nth-child(1) > div:nth-child(1) > div:nth-child(2) > span:nth-child(1)')
avoidance_max_speed = driver.find_element_by_css_selector('div.ratings-collapse:nth-child(2) > div:nth-child(2) > div:nth-child(1) > div:nth-child(4) > div:nth-child(3) > div:nth-child(1) > div:nth-child(1) > div:nth-child(2) > div:nth-child(1)').text[:-4]
wet_braking = data_point = driver.find_element_by_css_selector('div.ratings-collapse:nth-child(2) > div:nth-child(2) > div:nth-child(1) > div:nth-child(5) > div:nth-child(3) > div:nth-child(1) > div:nth-child(1) > div:nth-child(2) > div:nth-child(1)').text[:-4]
dry_braking = driver.find_element_by_css_selector('div.ratings-collapse:nth-child(2) > div:nth-child(2) > div:nth-child(1) > div:nth-child(5) > div:nth-child(3) > div:nth-child(1) > div:nth-child(2) > div:nth-child(2) > div:nth-child(1)').text[:-4]
headlights = driver.find_element_by_css_selector('div.ratings-collapse:nth-child(2) > div:nth-child(2) > div:nth-child(1) > div:nth-child(7) > div:nth-child(2) > div:nth-child(1) > div:nth-child(1) > div:nth-child(2) > span:nth-child(1)').text[:-1]
driver.find_element_by_xpath('/html/body/div[7]/div/div[2]/div[3]/h3/div[2]').click()
driving_position = driver.find_element_by_xpath('/html/body/div[7]/div/div[2]/div[3]/div/div/div[3]/div[2]/div/div/div[2]/span[1]').text[:-1]
front_shoulder_room = driver.find_element_by_xpath('/html/body/div[7]/div/div[2]/div[3]/div/div/div[4]/div[3]/div/div[1]/div[2]/div').text[:-4]
front_leg_room = driver.find_element_by_xpath('/html/body/div[7]/div/div[2]/div[3]/div/div/div[4]/div[3]/div/div[2]/div[2]/div').text[:-4]
front_head_room = driver.find_element_by_xpath('/html/body/div[7]/div/div[2]/div[3]/div/div/div[4]/div[3]/div/div[3]/div[2]/div').text[:-4]
rear_shoulder_room = driver.find_element_by_xpath('/html/body/div[7]/div/div[2]/div[3]/div/div/div[5]/div[3]/div/div[1]/div[2]/div').text[:-4]
rear_leg_room = driver.find_element_by_xpath('/html/body/div[7]/div/div[2]/div[3]/div/div/div[5]/div[3]/div/div[2]/div[2]/div').text[:-4]
rear_head_room = driver.find_element_by_xpath('/html/body/div[7]/div/div[2]/div[3]/div/div/div[5]/div[3]/div/div[3]/div[2]/div').text[:-4]
third_row_score = driver.find_element_by_xpath('/html/body/div[7]/div/div[2]/div[3]/div/div/div[6]/div[2]/div/div/div[2]').text[0:1]
third_shoulder_room = driver.find_element_by_xpath('/html/body/div[7]/div/div[2]/div[3]/div/div/div[6]/div[3]/div/div[1]/div[2]/div').text[0:-4]
third_leg_room = driver.find_element_by_xpath('/html/body/div[7]/div/div[2]/div[3]/div/div/div[6]/div[3]/div/div[2]/div[2]/div').text[0:-4]
third_head_room = driver.find_element_by_xpath('/html/body/div[7]/div/div[2]/div[3]/div/div/div[6]/div[3]/div/div[3]/div[2]/div').text[0:-4]
front_access = driver.find_element_by_xpath('/html/body/div[7]/div/div[2]/div[3]/div/div/div[7]/div[2]/div/div/div[2]/span[1]').text[0:1]
rear_access = driver.find_element_by_xpath('/html/body/div[7]/div/div[2]/div[3]/div/div/div[8]/div[2]/div/div/div[2]').text[0:1]
third_access = driver.find_element_by_xpath('/html/body/div[7]/div/div[2]/div[3]/div/div/div[9]/div[2]/div/div/div[2]').text[0:1]
usability = driver.find_element_by_xpath('/html/body/div[7]/div/div[2]/div[3]/div/div/div[10]/div[2]/div/div/div[2]/span[1]').text[0:1]
climate_system = driver.find_element_by_xpath('/html/body/div[7]/div/div[2]/div[3]/div/div/div[13]/div[2]/div/div/div[2]/span[1]').text[0:1]
driver.find_element_by_xpath('/html/body/div[7]/div/div[2]/div[4]/h3/div[2]').click()
fuel_capacity = driver.find_element_by_xpath('/html/body/div[7]/div/div[2]/div[4]/div/div/div[6]/div[2]/div').text[0:-5]
fuel_type = driver.find_element_by_xpath('/html/body/div[7]/div/div[2]/div[4]/div/div/div[7]/div[2]/div').text


In [300]:
third_row_score

'1'

In [70]:
#url = 'https://www.consumerreports.org/cars/kia/telluride/2020/ratings/?pagestop'
#driver.get(url)
#driver.find_element_by_xpath('/html/body/div[7]/div/div[2]/div[4]/h3/div[2]').click()
data_point = driver.find_element_by_xpath('/html/body/div[7]/div/div[2]/div[4]/div/div/div[7]/div[2]/div').text



In [71]:
data_point 





'Regular'

In [254]:
<div class="ratings-row clearfix"><div class="col-xs-12 col-sm-6 col-lg-4 crux-label-style ratings-row__label"><div>Acceleration</div></div><div class="col-xs-12 col-sm-6 col-lg-4 ratings-row__values"><div class="ratings-bar-chart"><div class="bar-ratings-chart-member"><div class="bar-ratings-chart-member__chart"><span class="bar-ratings-chart-member__rating crux-color-excellent-bg" style="width: 100%;"></span></div><div class="bar-ratings-chart-member__range"><span class="crux-body-copy crux-body-copy--extra-small--bold bar-ratings-chart-member__score">5&nbsp;</span><span class="crux-body-copy crux-body-copy--extra-small bar-ratings-chart-member__max-scale">/ 5</span></div></div></div></div><div><div><div class="ratings-sub-row clearfix"><div class="col-xs-12 col-sm-6 col-lg-4 ratings-sub-row__label"><div>0 to 30 MPH<span><span><span class="i-text-icon crux-icons crux-icons-help-information"></span></span></span></div></div><div class="col-xs-12 col-sm-6 col-lg-4 ratings-sub-row__values"><div class="crux-body-copy crux-body-copy--small compare-widget__type-text clearfix ">2.1 sec.</div></div></div><div class="ratings-sub-row clearfix"><div class="col-xs-12 col-sm-6 col-lg-4 ratings-sub-row__label"><div>0 to 60 MPH<span><span><span class="i-text-icon crux-icons crux-icons-help-information"></span></span></span></div></div><div class="col-xs-12 col-sm-6 col-lg-4 ratings-sub-row__values"><div class="crux-body-copy crux-body-copy--small compare-widget__type-text clearfix ">5.2 sec.</div></div></div><div class="ratings-sub-row clearfix"><div class="col-xs-12 col-sm-6 col-lg-4 ratings-sub-row__label"><div>45 to 65 MPH<span><span><span class="i-text-icon crux-icons crux-icons-help-information"></span></span></span></div></div><div class="col-xs-12 col-sm-6 col-lg-4 ratings-sub-row__values"><div class="crux-body-copy crux-body-copy--small compare-widget__type-text clearfix ">3.6 sec.</div></div></div><div class="ratings-sub-row clearfix"><div class="col-xs-12 col-sm-6 col-lg-4 ratings-sub-row__label"><div>Quarter-Mile<span><span><span class="i-text-icon crux-icons crux-icons-help-information"></span></span></span></div></div><div class="col-xs-12 col-sm-6 col-lg-4 ratings-sub-row__values"><div class="crux-body-copy crux-body-copy--small compare-widget__type-text clearfix ">13.8 sec.</div></div></div><div class="ratings-sub-row clearfix"><div class="col-xs-12 col-sm-6 col-lg-4 ratings-sub-row__label"><div>Quarter-Mile<span><span><span class="i-text-icon crux-icons crux-icons-help-information"></span></span></span></div></div><div class="col-xs-12 col-sm-6 col-lg-4 ratings-sub-row__values"><div class="crux-body-copy crux-body-copy--small compare-widget__type-text clearfix ">105 mph</div></div></div></div></div></div>

'<html lang="en-US" class="js wwx-flexbox wwx-canvas wwx-canvastext wwx-webgl no-touch wwx-geolocation wwx-postmessage wwx-websqldatabase wwx-indexeddb wwx-hashchange wwx-history wwx-draganddrop wwx-websockets wwx-rgba wwx-hsla wwx-multiplebgs wwx-backgroundsize wwx-borderimage wwx-borderradius wwx-boxshadow wwx-textshadow wwx-opacity wwx-cssanimations wwx-csscolumns wwx-cssgradients wwx-cssreflections wwx-csstransforms wwx-csstransforms3d wwx-csstransitions wwx-fontface wwx-generatedcontent wwx-video wwx-audio wwx-localstorage wwx-sessionstorage wwx-webworkers wwx-applicationcache wwx-svg wwx-inlinesvg wwx-smil wwx-svgclippaths"><head><style type="text/css" class="bc-style-2ab51b8c-6ce0-4c90-beb2-9a8912cc085e-default">.bc-iframe,.bc-iframe body{height:100%;margin:0;padding:0;width:100%;overflow:hidden}.bc-iframe body,.bc-player-2ab51b8c-6ce0-4c90-beb2-9a8912cc085e_default{background-color:#151B17}.bc-iframe .bc-player-2ab51b8c-6ce0-4c90-beb2-9a8912cc085e_default{background-color:trans

In [282]:
url = 'https://www.consumerreports.org/cars/toyota/corolla/2011/owner-satisfaction/'
driver.get(url)
driving_satisfaction = driver.find_element_by_xpath('/html/body/div[7]/section/article[1]/div/div/div[2]/div/div/div/div[2]').text[0:2]
comfort_satisfaction = driver.find_element_by_xpath('/html/body/div[7]/section/article[2]/div/div/div[2]/div/div/div/div[2]').text[0:2]
value_satisfaction = driver.find_element_by_xpath('/html/body/div[7]/section/article[3]/div/div/div[2]/div/div/div/div[2]').text[0:2]
styling_satisfaction = driver.find_element_by_xpath('/html/body/div[7]/section/article[4]/div/div/div[2]/div/div/div/div[2]').text[0:2]

In [285]:
value_satisfaction

'82'

# Scrape Annual Sales

In [286]:
styling_satisfaction

'47'